In [1]:
# !pip install kmodes
# !pip install plotnine

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes
# Import module for data visualization
from plotnine import *
import plotnine

In [3]:
df_2=pd.read_csv("rec_sys_alumnos.csv")
df_2['edad']=pd.to_numeric(df_2['edad'].map(lambda x: x.replace(' ','') if type(x)==str else x),errors='coerce')
df_2['num_antiguedad']=pd.to_numeric(df_2['num_antiguedad'].map(lambda x: x.replace(' ','') if type(x)==str else x),errors='coerce')
df_2.fecha1=df_2.fecha1.map(lambda x:x[:4] if x is not np.nan else np.nan).astype(float)
dict_mes = dict(zip(df_2.mes.unique(),range(1,17)))
df_2.mes = df_2.mes.map(dict_mes).astype('int')

C:\Users\ICS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5,9,11) have mixed types.Specify dtype option on import or set low_memory=False.


Para la variable _fecha1_ , nos quedamos solo con el año, que es lo que nos interesa.

In [4]:
columnas = df_2.columns.drop(['Unnamed: 0','xti_nuevo_cliente','num_antiguedad','mes','xti_rel','fec_ult_cli_1t',
                  'xti_rel_1mes','tip_rel_1mes','xti_extra','ind_prod1', 'ind_prod2',
       'ind_prod3', 'ind_prod4', 'ind_prod5', 'ind_prod6', 'ind_prod7',
       'ind_prod8', 'ind_prod9', 'ind_prod10', 'ind_prod11', 'ind_prod12',
       'ind_prod13', 'ind_prod14', 'ind_prod15', 'ind_prod16', 'ind_prod17',
       'ind_prod18', 'ind_prod19', 'ind_prod20', 'ind_prod21', 'ind_prod22',
       'ind_prod23', 'ind_prod24', 'ind_prod25'])

In [5]:
df_2[columnas].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634682 entries, 0 to 634681
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   cod_persona            634682 non-null  int64  
 1   pais                   634626 non-null  object 
 2   sexo                   634617 non-null  object 
 3   edad                   634626 non-null  float64
 4   fecha1                 634626 non-null  float64
 5   xti_empleado           634626 non-null  object 
 6   indresi                634626 non-null  object 
 7   indext                 634626 non-null  object 
 8   des_canal              627504 non-null  object 
 9   tip_dom                634626 non-null  float64
 10  cod_provincia          631736 non-null  float64
 11  xti_actividad_cliente  634626 non-null  float64
 12  imp_renta              510518 non-null  float64
 13  id_segmento            627279 non-null  object 
 14  mean_engagement        629040 non-nu

In [6]:
columnas

Index(['cod_persona', 'pais', 'sexo', 'edad', 'fecha1', 'xti_empleado',
       'indresi', 'indext', 'des_canal', 'tip_dom', 'cod_provincia',
       'xti_actividad_cliente', 'imp_renta', 'id_segmento', 'mean_engagement'],
      dtype='object')

Hacemos un clustering de clientes utilizando las variables que no dependen del tiempo. Para ello agrupamos por clientes y nos quedamos con la moda de las variables, ya que así podemos rellenar una gran cantidad de valores faltantes.

In [7]:
df_clustering = df_2[columnas].groupby('cod_persona').agg(dict({k:(lambda x: pd.Series.mode(x) if x.unique()[0] is not np.nan else np.nan) for k in 
                                                                [  'cod_provincia']},
                                         **{'edad':'max','mean_engagement':'mean','imp_renta':'mean'},
                                         **{k:(lambda x: pd.Series.mode(x)[0] if x.unique()[0] is not np.nan else np.nan) for k in 
                                            ['id_segmento','des_canal','indresi','pais', 'xti_actividad_cliente','xti_empleado','sexo','fecha1','indext','tip_dom' ] }))

Las columnas _id_segmento_ y _des_canal_ dan problemas porque hay clientes con todos los datos nulos en esas columnas, luego el agg no puede dar el valor de la moda. Por eso hay que especificar que ponga un nan.
Para quitarnos los valores nulos que nos quedan, imputamos por la moda o la mediana directamente.


In [8]:
df_clustering.cod_provincia = df_clustering.cod_provincia.map(lambda x: x if isinstance(x,float) else np.nan)

In [9]:
df_clustering.cod_provincia.fillna(df_clustering.cod_provincia.mode()[0],inplace=True)

In [10]:
for col in ['mean_engagement','imp_renta']:
    df_clustering[col]=df_clustering[col].fillna(df_clustering[col].mean())
    
for col in ['pais','xti_empleado','sexo','indresi','indext','id_segmento','des_canal']:
    df_clustering[col]=df_clustering[col].fillna(df_clustering[col].mode()[0])


In [11]:
df_clustering.tip_dom.unique()

array([1.])

In [12]:
# Esta variable es constante, luego no nos aporta nada. La quitamos.
df_clustering.drop('tip_dom', axis=1,inplace=True)

In [13]:
df_clustering.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46779 entries, 5514 to 1537800
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cod_provincia          46779 non-null  float64
 1   edad                   46779 non-null  float64
 2   mean_engagement        46779 non-null  float64
 3   imp_renta              46779 non-null  float64
 4   id_segmento            46779 non-null  object 
 5   des_canal              46779 non-null  object 
 6   indresi                46779 non-null  object 
 7   pais                   46779 non-null  object 
 8   xti_actividad_cliente  46779 non-null  float64
 9   xti_empleado           46779 non-null  object 
 10  sexo                   46779 non-null  object 
 11  fecha1                 46779 non-null  float64
 12  indext                 46779 non-null  object 
dtypes: float64(6), object(7)
memory usage: 5.0+ MB


In [14]:
df_clustering['pais'].value_counts()

ES    46593
DE       18
GB       16
FR       14
US       13
CH       12
CO       10
IT        9
AR        9
MX        8
RO        6
EC        6
CL        5
PY        5
VE        5
BR        4
BE        4
AT        3
CA        3
NL        2
PE        2
FI        2
BO        2
PL        2
BG        2
CN        2
UA        1
NG        1
PR        1
AU        1
PK        1
HU        1
PA        1
SE        1
IL        1
MD        1
NZ        1
IN        1
RU        1
CU        1
JP        1
PT        1
AD        1
SV        1
GE        1
GN        1
GT        1
SA        1
Name: pais, dtype: int64

Vamos a crear una nueva varaible que sea 1 si el país es España y que sea 0 en otro caso, porque la inmensa mayoría son de España. 

In [15]:
df_clustering.pais = df_clustering.pais.map(lambda x: 1 if x=='ES' else 0)

In [16]:
df_clustering.sexo.value_counts()

V    25490
H    21289
Name: sexo, dtype: int64

In [17]:
dict_sexo = {'H':0,'V':1}
df_clustering.sexo = df_clustering.sexo.map(dict_sexo).astype('int')

In [19]:
df_clustering.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46779 entries, 5514 to 1537800
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cod_provincia          46779 non-null  float64
 1   edad                   46779 non-null  float64
 2   mean_engagement        46779 non-null  float64
 3   imp_renta              46779 non-null  float64
 4   id_segmento            46779 non-null  object 
 5   des_canal              46779 non-null  object 
 6   indresi                46779 non-null  object 
 7   pais                   46779 non-null  int64  
 8   xti_actividad_cliente  46779 non-null  float64
 9   xti_empleado           46779 non-null  object 
 10  sexo                   46779 non-null  int32  
 11  fecha1                 46779 non-null  float64
 12  indext                 46779 non-null  object 
dtypes: float64(6), int32(1), int64(1), object(5)
memory usage: 4.8+ MB


In [20]:
categ = [0,4,5,6,7,8,9,10,12]

In [ ]:
# Choose optimal K using Elbow method
cost = []
kprototype =[0,0,0,0,0,0,0,0,0,0,0,0]
clusters = [0,0,0,0,0,0,0,0,0,0,0,0]

for cluster in range(1, 12):
    try:
        kprototype[cluster] = KPrototypes(n_jobs = -1, n_clusters = cluster, init = 'Huang', random_state = 0)
        clusters[cluster] = kprototype[cluster].fit_predict(df_clustering, categorical = categ)
        cost.append(kprototype[cluster].cost_)
        print('Cluster initiation: {}'.format(cluster))
    except:
        break

In [28]:
df_cost = pd.DataFrame({'Cluster':range(1, len(cost)+1), 'Cost':cost})

In [ ]:
df_cost

In [ ]:
plt.plot(df_cost.Cluster, df_cost.Cost, color='green', linestyle='dashed', marker='o',
     markerfacecolor='green', markersize=12)

In [33]:
clusterizacion = pd.DataFrame(data={'cod_persona':df_clustering.index,'cluster':clusters[5]})
clusterizacion.to_csv('clustering_5.csv', index=False)

In [ ]:
clusterizacion